#Announcement

I made a big switch here. In video I use the Pima dataset from Midterm 1. But I decided it was too confusing to start messing with this dataset during the final project. It seemed a lot more consistent to stick with Titanic, so that is what I did. Steps are the same, with caveat below.

I made a change to  *Feature reduction*. I have given you a new way (new library) that is more useful than the pandas `.corr` method.


I also removed the need to save the test set. We really do not need it.

Finally, I am asking you to build the markdown (md) desription of your pipeline including a screenshot of it. Refer back end to Chapter 7 for a model of what you need. You may be able to copy and paste what I have in Chapter 7 then edit it to fit your new pipeline.

<center>
<h1>First Notebook for Final Project (Wrangling)</h1>
</center>

<hr>

I am using Titanic dataset. You will need to fill in with your own. So replace my csv, random_state and pipeline with your own.

Reminder: here is a list of places you can find data to use: https://careerfoundry.com/en/blog/data-analytics/where-to-find-free-datasets/.

#Notes on your choice of dataset

* I would like you to find a csv dataset that has a mixture of numeric and categorical columns. So you can show off your transformers!

* I'd like a binary label column, i.e., what you will predict.

* Do not worry about number of rows. If they are below 500 you can use SMOTE to increase them. If they are over 5000, you can use downsample to decrease them.

* Don't worry about number of feature columns. I can help you reduce a large number down to 10 or less.


#I'd like to know the dataset you choose. Email me a link.

Do this before starting this notebook. I may have some guidance that will help you and potentially direct you off of dead-end paths.

##Set-up

First bring in your library.

In [98]:
github_name = 'kwoeser'
repo_name = 'CS423'
source_file = 'library.py'
url = f'https://raw.githubusercontent.com/{github_name}/{repo_name}/main/{source_file}'
!rm $source_file
!wget $url
%run -i $source_file

--2025-06-09 03:12:30--  https://raw.githubusercontent.com/kwoeser/CS423/main/library.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32297 (32K) [text/plain]
Saving to: ‘library.py’

library.py          100%[===================>]  31.54K  --.-KB/s    in 0.01s   

2025-06-09 03:12:30 (2.74 MB/s) - ‘library.py’ saved [32297/32297]



##Demo with Titanic

##Caveat

You will not match my results. You have your own dataset. I am just giving you the outline of the steps I expect you to follow.

But you will need your dataset stored in a way that you can load it (and I can load it). This may mean downloading from web (e.g., kaggle) and then uploading to GitHub.

In [99]:
url = 'https://raw.githubusercontent.com/kwoeser/CS423/main/final/data/heart.csv'


heart = pd.read_csv(url)
heart.head()


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [100]:
len(heart)

918

#Break out into features and labels



In [101]:
# Split into features and labels

features = heart.drop(columns='HeartDisease')
labels = heart['HeartDisease'].tolist()

In [102]:
labels.count(1)/len(labels)

0.5533769063180828

#Downsampling (optional)

Here is code to reduce the number of rows while keeping the new table stratified on the target column.

You can skip this step if your table has less than 5000 rows. You could probably skip it with less than 10000. The whole point is to avoid tuning (next notebook) that takes hours. In a real setting, you would probably have to bite the bullet and wait that long.

Note that I am setting N=1000 just for demo purposes. You should use 5000 or even greater as your value if you do need to downsample.

In [103]:
import lightgbm as lgb
import pandas as pd

temp_features_df = features.copy()

categorical_features = ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']

for col in categorical_features:
    temp_features_df[col] = temp_features_df[col].astype('category')

lgbm = lgb.LGBMClassifier()
lgbm.fit(temp_features_df, labels, categorical_feature=categorical_features)

# Show feature importance
feature_importance_series = pd.Series(lgbm.feature_importances_, index=temp_features_df.columns)
print("Feature Importance Ranking:")
print(feature_importance_series.sort_values(ascending=False))


[LightGBM] [Info] Number of positive: 508, number of negative: 410
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000146 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 403
[LightGBM] [Info] Number of data points in the train set: 918, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.553377 -> initscore=0.214324
[LightGBM] [Info] Start training from score 0.214324
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Feature Importance Ranking:
Cholesterol       630
MaxHR             509
Age               449
RestingBP         402
Oldpeak           393
ST_Slope          141
RestingECG        135
Sex               107
ExerciseAngina    103
FastingBS          79
ChestPainType      51
dtype: int32


In [104]:
N = 900
target = 'HeartDisease'
original_df = heart.copy()

downsample_df = original_df.groupby(target, group_keys=False).apply(lambda x: x.sample(int(np.rint(N * len(x)/len(original_df))))).sample(frac=1).reset_index(drop=True)

print(downsample_df[target].value_counts())
downsample_df


HeartDisease
1    498
0    402
Name: count, dtype: int64


<ipython-input-104-dca416beb743>:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  downsample_df = original_df.groupby(target, group_keys=False).apply(lambda x: x.sample(int(np.rint(N * len(x)/len(original_df))))).sample(frac=1).reset_index(drop=True)


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,63,M,ATA,136,165,0,ST,133,N,0.2,Up,0
1,66,M,ASY,160,228,0,LVH,138,N,2.3,Up,0
2,54,F,ATA,120,221,0,Normal,138,N,1.0,Up,0
3,58,M,ATA,130,230,0,Normal,150,N,0.0,Up,0
4,55,M,ASY,122,223,1,ST,100,N,0.0,Flat,1
...,...,...,...,...,...,...,...,...,...,...,...,...
895,61,M,ASY,150,0,0,Normal,117,Y,2.0,Flat,1
896,54,M,ASY,130,0,0,ST,117,Y,1.4,Flat,1
897,37,M,ASY,120,223,0,Normal,168,N,0.0,Up,0
898,48,M,ASY,160,193,0,Normal,102,Y,3.0,Flat,1


In [105]:
# 4/3 numeric/categorical split
selected_features = ['Cholesterol', 'MaxHR', 'Age', 'RestingBP', 'Sex', 'ST_Slope', 'RestingECG']

features_reduced = downsample_df[selected_features]
labels_reduced = downsample_df['HeartDisease']

features_reduced.head()

,Cholesterol,MaxHR,Age,RestingBP,Sex,ST_Slope,RestingECG
0,165,133,63,136,M,Up,ST
1,228,138,66,160,M,Up,LVH
2,221,138,54,120,F,Up,Normal
3,230,150,58,130,M,Up,Normal
4,223,100,55,122,M,Flat,ST


In [106]:
downsample_df['HeartDisease'].to_list().count(1)/len(downsample_df)  #same as original so seems to work

0.5533333333333333

###For those who want to know

Here is brief explanation:

* `original_df.groupby(target, group_keys=False)`: This groups the dataframe by the target column. The `group_keys=False` ensures that the original grouping columns are not included in the resulting dataframe.

* The apply method is used to apply a function to each group:

 * `lambda x: x.sample(int(np.rint(N*len(x)/len(original_df))))`: This function samples from each group based on the ratio of the group size to the total dataframe size multiplied by N (the number of samples you want from the entire dataframe).
 * The `np.rint` function rounds the number to the nearest integer.
 * `sample(frac=1)`: This shuffles the rows of the resulting dataframe.

* reset_index(drop=True): This resets the index of the dataframe and drops the original index.

In essence, the code calculates the number of samples needed from each group directly, based on the proportion of the group in the original dataframe, and then samples that number of rows from each group. It then shuffles the combined downsampled dataframe.

#Feature reduction (optional)

Some will need to drop columns. I'll give you a new library.  I like it because it works with a table with a mixture of datatypes, e.g., categorical columns.

You should be able to use `correlation_df` (below) in place of the code you worked on earlier that used the pandas correlation table.

You can play around with the threshold to get to your target number of features.



###New correlation method

The `titanic_trimmed.corr()` given  will give you an error if you have mixed column types. This new method handles this fine. When comparing two columns, it uses:

* Cramer's V for categorical-categorical.
* correlation ratio for numeric-categorical.
* Pearson for numeric-numeric.

##If you downsample and/or do feature reduction

Save the resulting table out to github so you do not have to keep repeating the process.


In [107]:
# saves reduced csv file with top 7 features
final_df = features_reduced.copy()
final_df['HeartDisease'] = labels_reduced

final_df.to_csv("heart_reduced.csv", index=False)

In [108]:
# Load the reduced dataset
reduced_lc_df = pd.read_csv("heart_reduced.csv")

# Split into features and labels
features = reduced_lc_df.drop(columns='HeartDisease')
labels = reduced_lc_df['HeartDisease']


In [109]:
features

,Cholesterol,MaxHR,Age,RestingBP,Sex,ST_Slope,RestingECG
0,165,133,63,136,M,Up,ST
1,228,138,66,160,M,Up,LVH
2,221,138,54,120,F,Up,Normal
3,230,150,58,130,M,Up,Normal
4,223,100,55,122,M,Flat,ST
...,...,...,...,...,...,...,...
895,0,117,61,150,M,Flat,Normal
896,0,117,54,130,M,Flat,ST
897,223,168,37,120,M,Up,Normal
898,193,102,48,160,M,Flat,Normal


#Define your pipeline

This is up to you for your own dataset.


In [110]:
heart_transformer = Pipeline(steps=[
    ('tukey_cholesterol', CustomTukeyTransformer('Cholesterol', 'outer')),
    ('tukey_maxhr', CustomTukeyTransformer('MaxHR', 'outer')),
    ('tukey_age', CustomTukeyTransformer('Age', 'outer')),
    ('tukey_restbp', CustomTukeyTransformer('RestingBP', 'outer')),
    ('scale_cholesterol', CustomRobustTransformer('Cholesterol')),
    ('scale_maxhr', CustomRobustTransformer('MaxHR')),
    ('scale_age', CustomRobustTransformer('Age')),
    ('scale_restbp', CustomRobustTransformer('RestingBP')),
    ('ohe_sex', CustomOHETransformer('Sex')),
    ('ohe_st_slope', CustomOHETransformer('ST_Slope')),
    ('ohe_resting_ecg', CustomOHETransformer('RestingECG')),
    ('imputer', CustomKNNTransformer(n_neighbors=5))
], verbose=True)

##Take a screenshot of your pipeline

Save it on GitHub. Then use Chapter 7 markdown example to build your own pipeline description. Also save this on GitHub.

##Find random state for splitting



In [111]:
heart_transformed_df = heart_transformer.fit_transform(features, labels)

[Pipeline]  (step 1 of 12) Processing tukey_cholesterol, total=   0.0s
[Pipeline] ...... (step 2 of 12) Processing tukey_maxhr, total=   0.0s
[Pipeline] ........ (step 3 of 12) Processing tukey_age, total=   0.0s
[Pipeline] ..... (step 4 of 12) Processing tukey_restbp, total=   0.0s
[Pipeline]  (step 5 of 12) Processing scale_cholesterol, total=   0.0s
[Pipeline] ...... (step 6 of 12) Processing scale_maxhr, total=   0.0s
[Pipeline] ........ (step 7 of 12) Processing scale_age, total=   0.0s
[Pipeline] ..... (step 8 of 12) Processing scale_restbp, total=   0.0s


[Pipeline] .......... (step 9 of 12) Processing ohe_sex, total=   0.0s


[Pipeline] .... (step 10 of 12) Processing ohe_st_slope, total=   0.0s


[Pipeline] . (step 11 of 12) Processing ohe_resting_ecg, total=   0.0s
[Pipeline] ......... (step 12 of 12) Processing imputer, total=   0.0s


In [112]:
heart_transformed_df.head()

,Cholesterol,MaxHR,Age,RestingBP,Sex_F,Sex_M,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up,RestingECG_LVH,RestingECG_Normal,RestingECG_ST
0,-0.603675,-0.142857,0.692308,0.3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.057743,0.000000,0.923077,1.5,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
2,-0.015748,0.000000,0.000000,-0.5,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.078740,0.342857,0.307692,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.005249,-1.085714,0.076923,-0.4,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0


#Find random_state value



In [118]:
%%capture
rs, _ = find_random_state(features, labels, heart_transformer)

In [119]:
rs  #whatever it is for your own dataset

np.int64(102)

###Remember this value for next notebook, tuning

##Split your dataset and save fitted pipeline

In [120]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=rs)
production_pipeline = heart_transformer.fit(X_train, y_train) #now ready for production


[Pipeline]  (step 1 of 12) Processing tukey_cholesterol, total=   0.0s
[Pipeline] ...... (step 2 of 12) Processing tukey_maxhr, total=   0.0s
[Pipeline] ........ (step 3 of 12) Processing tukey_age, total=   0.0s
[Pipeline] ..... (step 4 of 12) Processing tukey_restbp, total=   0.0s
[Pipeline]  (step 5 of 12) Processing scale_cholesterol, total=   0.0s
[Pipeline] ...... (step 6 of 12) Processing scale_maxhr, total=   0.0s
[Pipeline] ........ (step 7 of 12) Processing scale_age, total=   0.0s
[Pipeline] ..... (step 8 of 12) Processing scale_restbp, total=   0.0s


[Pipeline] .......... (step 9 of 12) Processing ohe_sex, total=   0.0s


[Pipeline] .... (step 10 of 12) Processing ohe_st_slope, total=   0.0s


[Pipeline] . (step 11 of 12) Processing ohe_resting_ecg, total=   0.0s
[Pipeline] ......... (step 12 of 12) Processing imputer, total=   0.0s


In [123]:
import joblib

joblib.dump(production_pipeline, "final_fully_fitted_pipeline.pkl")  #Move this to GitHub where you will use it in production

['final_fully_fitted_pipeline.pkl']

In [124]:
ptransformer = joblib.load("final_fully_fitted_pipeline.pkl")  # Make sure can get it back


#Congrats

You are done with the wrangling stage. Ready to move on to training and tuning models.

#Reminder

You should have the following files stored on GitHub.

1. Screenshot of pipeline.

2. Markdown pipeline description that references screenshot.

3. `final_fully_fitted_pipeline.pkl`

4. Value for `rs` to use in train_test_split. Saving this to GitHub is optional. You can just remember it and use it in tuning notebook.